# Imports

In [ ]:
import urllib.request
import os
import patoolib
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import glob,os
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Selenium

In [ ]:
download_path = os.path.abspath(os.path.join(os.path.abspath(os.getcwd()), os.pardir)) + '\\data\\raw\\covid_saude'
options = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_path}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(executable_path='./chromedriver',chrome_options=options)

try:
    driver.get("https://covid.saude.gov.br/")
    # delay = 3 # seconds
    # try:
    #     myElem = WebDriverWait(browser, delay).until(EC.element_to_be_clickable((By.CLASS_NAME, "btn-white.md.button.button-solid.button-has-icon-only.ion-activatable.ion-focusable.hydrated")))
    #     print("Page is ready!")
    # except Exception as e:
    #     print("Loading took too much time!")

    button = driver.find_element_by_class_name("btn-white.md.button.button-solid.button-has-icon-only.ion-activatable.ion-focusable.hydrated")
    time.sleep(3)
    button.click()
    time.sleep(10)
    driver.close()

    # Delete old .csv files
    dir_name = '../data/raw/covid_saude'

    # Unzip files and delete zipped file
    for item in os.listdir(dir_name): # loop through items in dir
        if item.endswith('.rar'): # check for ".rar" extension
            file_name = os.path.abspath(item) # get full path of files
            file_name = file_name.replace('notebooks', 'data\\raw\\covid_saude')
            patoolib.extract_archive(file_name, outdir="../data/raw/covid_saude")
            time.sleep(3)
            os.remove(file_name) # delete zipped file
    
except Exception as e:
    print('Error:', e)

# Data Processing

In [ ]:
list_dfs = []

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith('.csv'): # check for ".csv" extension
        list_dfs.append(pd.read_csv('../data/raw/covid_saude/' + item, sep=";"))
        
df_covid_saude = pd.concat(list_dfs)

df_covid_saude.to_parquet("../data/processed/covid_saude.parquet", index=False)

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith('.csv'): # check for ".csv" extension
        file_name = os.path.abspath(item) # get full path of files
        file_name = file_name.replace('notebooks', 'data\\raw\\covid_saude')
        os.remove(file_name) # delete zipped file